# Data model
## Installation

The new data model currently lives in the branch `data-model`. You will have to install that version of PMD, which you can do by executing the code below.

In [2]:
]add /Users/sclaeys/code/PowerModelsDistribution.jl#data-model

In [ ]:
using PowerModelsDistribution

Before, you could import data from one of three sources:
- OpenDSS input files
- balanced matpower .m files, converted to unbalanced by PMs utilities
- unbalanced .m files, custom extension of the matpower format

We decided to drop support for the matpower extension. Instead, we decided to add a high-level, user-friendly data model directly in PMD. The OpenDSS parser will be updated to parse to this high-level data format. Furthermore, we defined a lot of utility functions which allow you to build your case study step-by-step, much like you can do it in OpenDSS.

## Build your own case

Start by creating an empty data model. For now this is still a regular Dict, but it might be wrapped in a struct in the future.

In [17]:
data_model = create_data_model()

Dict{String,Any} with 1 entry:
  "v_var_scalar" => 1000.0

First, we will add a voltage source to our network. Each component can be added by a similar syntax. First, you pass the data model you want to add the component to, and then you specify the properties of that component through keywords arguments. 
- **id**: This is an identifier for the component you create, which must be unique amongst components of the same type. This will be the key for the component dictionaries at the high-level (both for the data model and the solutions). The id can be anything (as in any type) you want, but if you want to save it as a json file, Strings or Integers are recommended.

Most values are entered in SI units (TODO: explain the scalar thing).

### Buses, voltage sources, linecodes and lines

In [18]:
add_bus!(data_model, id=:sourcebus, terminals=[:a, :b, :c, :n])
add_bus!(data_model, id=:loadbus,   terminals=[:a, :b, :c, :n])

Dict{String,Any} with 6 entries:
  "rg"        => Float64[]
  "grounded"  => Any[]
  "status"    => 1
  "terminals" => Symbol[:a, :b, :c, :n]
  "id"        => :loadbus
  "xg"        => Float64[]

In [19]:
add_voltage_source!(data_model, id=:source, vm=[0.23, 0.23, 0.23, 0], va=[0, -2*pi/3, 2*pi/3, 0],
    bus=:sourcebus, connections=[:a, :b, :c, :n]
)

Dict{String,Any} with 6 entries:
  "va"          => [0.0, -2.0944, 2.0944, 0.0]
  "status"      => 1
  "connections" => Symbol[:a, :b, :c, :n]
  "id"          => :source
  "vm"          => [0.23, 0.23, 0.23, 0.0]
  "bus"         => :sourcebus

In [20]:
add_linecode!(data_model, id=:lc_3w, rs=ones(3,3), xs=ones(3,3))

Dict{String,Any} with 7 entries:
  "b_fr" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "rs"   => [1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
  "id"   => :lc_3w
  "xs"   => [1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
  "g_to" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "b_to" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "g_fr" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

In [21]:
add_line!(data_model, id=:line1, linecode=:lc_3w, length=1.2, 
    f_bus=:sourcebus, f_connections=[:a, :b, :c], t_bus=:loadbus, t_connections=[:a, :b, :c],
)

Dict{String,Any} with 10 entries:
  "length"        => 1.2
  "t_connections" => Symbol[:a, :b, :c]
  "f_bus"         => :sourcebus
  "angmin"        => [-1.0472, -1.0472, -1.0472]
  "status"        => 1
  "angmax"        => [1.0472, 1.0472, 1.0472]
  "id"            => :line1
  "linecode"      => :lc_3w
  "t_bus"         => :loadbus
  "f_connections" => Symbol[:a, :b, :c]

### Transformer

First, we add a bus for the secondary of the transformer to connect to (you can also do this after creating the transformer, as long as you add it before checking the data model).
We start with a two-winding, three-phase, lossless, delta-wye transformer.

In [22]:
add_bus!(data_model, id=:tr_sec,   terminals=[:a, :b, :c, :n], grounded=[:n], rg=[0.0], xg=[0.0])
add_transformer_nw!(data_model, id=:tr1,
    bus=[:loadbus, :tr1_sec], configuration=["wye", "delta"], connections=[[:a, :b, :c, :n], [:a, :b, :c]],
    vnom=[0.230, 0.230/2, 0.230/2]
)

Dict{String,Any} with 16 entries:
  "polarity"      => Bool[true, true]
  "connections"   => Array{Symbol,1}[[:a, :b, :c, :n], [:a, :b, :c]]
  "tm_min"        => Array{Float64,1}[[0.9, 0.9, 0.9], [0.9, 0.9, 0.9]]
  "tm_step"       => Array{Float64,1}[[0.03125, 0.03125, 0.03125], [0.03125, 0.…
  "bus"           => Symbol[:loadbus, :tr1_sec]
  "configuration" => ["wye", "delta"]
  "status"        => 1
  "id"            => :tr1
  "noloadloss"    => 0.0
  "xsc"           => [0.0, 0.0]
  "tm_fix"        => Array{Bool,1}[[true, true, true], [true, true, true]]
  "tm"            => Array{Float64,1}[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
  "tm_max"        => Array{Float64,1}[[1.1, 1.1, 1.1], [1.1, 1.1, 1.1]]
  "rs"            => [0.0, 0.0]
  "imag"          => 0.0
  "vnom"          => [0.23, 0.115, 0.115]

Next, we define a lossy split-phase transformer. Note that this can be seen as a single-phase, three-winding transformer with the appropriate parameters. In the future, we might define a component especially for split-phase transformers, so you do not have to map this manually.

In [23]:
add_bus!(data_model, id=:split_sec,   terminals=[:a, :b, :n], grounded=[:n], rg=[0.0], xg=[0.0])
add_transformer_nw!(data_model, id=:split,
    bus=[:loadbus, :split_sec, :split_sec], configuration=["wye", "wye", "wye"], connections=[[:a, :b], [:a, :n], [:n, :b]],
    vnom=[0.230, 0.230/2, 0.230/2], snom=[2, 1, 1],
    rs=[0.02, 0.01, 0.01],
    xsc=[0.02, 0.01, 0.01]
)

Dict{String,Any} with 17 entries:
  "polarity"      => Bool[true, true, true]
  "connections"   => Array{Symbol,1}[[:a, :b], [:a, :n], [:n, :b]]
  "tm_min"        => Array{Float64,1}[[0.9, 0.9, 0.9], [0.9, 0.9, 0.9], [0.9, 0…
  "tm_step"       => Array{Float64,1}[[0.03125, 0.03125, 0.03125], [0.03125, 0.…
  "bus"           => Symbol[:loadbus, :split_sec, :split_sec]
  "configuration" => ["wye", "wye", "wye"]
  "status"        => 1
  "id"            => :split
  "noloadloss"    => 0.0
  "xsc"           => [0.02, 0.01, 0.01]
  "snom"          => [2, 1, 1]
  "tm_fix"        => Array{Bool,1}[[true, true, true], [true, true, true], [tru…
  "tm"            => Array{Float64,1}[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1…
  "tm_max"        => Array{Float64,1}[[1.1, 1.1, 1.1], [1.1, 1.1, 1.1], [1.1, 1…
  "rs"            => [0.02, 0.01, 0.01]
  "imag"          => 0.0
  "vnom"          => [0.23, 0.115, 0.115]

### Loads

A load's voltage-dependency defaults to `model="constant_power"`, in which case you can specify its behaviour with `pd` and `qd`.

In [24]:
add_load!(data_model, id=:cp_y_2ph, pd=[0.1, 0.2], 
    bus=:loadbus, connections=[:a, :b, :n]
)

Dict{String,Any} with 8 entries:
  "status"        => 1
  "model"         => "constant_power"
  "connections"   => Symbol[:a, :b, :n]
  "id"            => :cp_y_2ph
  "qd"            => [0.0, 0.0, 0.0]
  "bus"           => :loadbus
  "pd"            => [0.1, 0.2]
  "configuration" => "wye"

A load can also be delta-connected.

In [25]:
add_load!(data_model, id=:cp_d_3ph, pd=[0.1, 0.2, 0.3], 
    bus=:loadbus, configuration="delta", connections=[:a, :b, :c]
)

Dict{String,Any} with 8 entries:
  "status"        => 1
  "model"         => "constant_power"
  "connections"   => Symbol[:a, :b, :c]
  "id"            => :cp_d_3ph
  "qd"            => [0.0, 0.0, 0.0]
  "bus"           => :loadbus
  "pd"            => [0.1, 0.2, 0.3]
  "configuration" => "delta"

Besides `constant_power`, other options are `constant_current`, `constant_impedance` and `exponential`.
For those options, you specify `pd_ref` and `qd_ref`, 
which will be obtained when the applied voltage is equal to the one specified by `vnom`.

In [26]:
add_load!(data_model, id=:cc_d_3ph, model="constant_current", pd_ref=[0.1, 0.2, 0.3], vnom=0.230*sqrt(3),
    bus=:loadbus, configuration="delta", connections=[:a, :b, :c]
)

Dict{String,Any} with 9 entries:
  "vnom"          => 0.398372
  "status"        => 1
  "model"         => "constant_current"
  "connections"   => Symbol[:a, :b, :c]
  "qd_ref"        => [0.0, 0.0, 0.0]
  "pd_ref"        => [0.1, 0.2, 0.3]
  "id"            => :cc_d_3ph
  "bus"           => :loadbus
  "configuration" => "delta"

For an exponential voltage-dependency load model, you also need to specify additionally `alpha` and `beta`.

In [27]:
add_load!(data_model, id=:exp_d_3ph, model="exponential", pd_ref=[0.1, 0.2, 0.3], vnom=0.230*sqrt(3),
    bus=:loadbus, configuration="delta", connections=[:a, :b, :c], alpha=[1.9, 2.0, 2.1], beta=[2.1, 2.0, 1.9]
)

Dict{String,Any} with 11 entries:
  "model"         => "exponential"
  "connections"   => Symbol[:a, :b, :c]
  "bus"           => :loadbus
  "configuration" => "delta"
  "status"        => 1
  "qd_ref"        => [0.0, 0.0, 0.0]
  "id"            => :exp_d_3ph
  "alpha"         => [1.9, 2.0, 2.1]
  "pd_ref"        => [0.1, 0.2, 0.3]
  "beta"          => [2.1, 2.0, 1.9]
  "vnom"          => 0.398372

### Generator

In [28]:
add_generator!(data_model, id=:gen1, cost=[1.0, 0],
    bus=:loadbus, connections=[:a, :b, :c, :n],
)

Dict{String,Any} with 6 entries:
  "cost"          => [1.0, 0.0]
  "status"        => 1
  "connections"   => Symbol[:a, :b, :c, :n]
  "id"            => :gen1
  "bus"           => :loadbus
  "configuration" => "wye"

### Checks

Finally, we can check whether the data model is consistent. There are many built in checks to keep you from making mistakes. For example, it will check that
- all buses and terminals you used are actually defined;
- all required data model properties are present;
- all properties you have defined have the right type (real, boolean, array, ...);
- arrays and vectors have the right size (the length of the `connections` property usually implies the 'right' size);
- more specialized checks at the component level.

When you execute the following, these checks will print a descriptive error message if something is wrong. You can try this out by going back and entering something inconsistent, i.e. connecting a load to an undefined bus.


In [ ]:
check_data_model(data_model)

## Transform to low-level data model

In [ ]:
data_model_map!(data_model)

In [ ]:
data_model_make_pu!(data_model)

In [ ]:
data_model_index!(data_model)

In [ ]:
data_model_make_compatible_v8!(data_model)

## Solve the low-level model

In [ ]:
import PowerModels
PMs = PowerModels
run_mc_pf(data_model, PMs.ACPPowerModel, build_mc_opf)

## Get high-level solution

In [ ]:
solution_identify!(sol["solution"])

In [ ]:
solution_unmake_pu!(sol["solution"])

In [ ]:
solution_unmap!(sol["solution"])